### Start from here


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
tf.keras.backend.set_floatx('float32')

In [2]:

# Load the data
data = pd.read_csv("C:\\Users\\Mohammad\\Desktop\\Uni\\Uni work\\Year 4\\Term 7, fall 2022\\CMPE 460 Deep Learning\project\\CMPE-460-Project\\dataset\\car_price_prediction.csv")

# drop rows where the price is less than 100
data = data[data['Price'] >= 100]
data = data[~data['Model'].isin(["GONOW","IVECO DAYLY"])]




# Drop the totalPrice column
data = data.drop('totalPrice', axis=1)
data = data.drop('Wheel', axis=1)
data = data.drop('no-tax', axis=1)
data = data.drop('Engine volume', axis=1)
data['Doors'] = data['Doors'].replace({'-May':'','-Mar':''}, regex=True)
data['Doors'] = data['Doors'].str.replace('>5','5')

# Drop rows with missing or empty values in the "Doors" column
data = data[data['Doors'].str.strip() !='']
data['Doors'] = data['Doors'].astype(int)

data['Doors'] = data['Doors'].astype(int)

# Gear box editing
data['Gear box type'] = data['Gear box type'].replace({'Automatic': 1, 'Tiptronic': 2,'Variator':3, 'Manual':4})
data = data[data['Gear box type'].isin([1,2,3,4])]

data["Gear box type"] = data["Gear box type"].astype(int)

# 'Automatic' is changed to 1
# 'Tiptronic' is changed to 2
# 'Variator' is changed to 3
# 'Manual' is changed to 4


# Replace values in the "Leather interior" column based on conditions
data["Leather interior"] = data["Leather interior"].replace({"Yes": 1, "No": 0}) # Yes is 1; No is 0
# keep the rows where the value of "Leather interior" is 0 or 1
data = data[data["Leather interior"].apply(lambda x: x in [0, 1])]



# fuel type edit
data["Fuel type"] = data["Fuel type"].replace({"Hybrid":1, "Petrol":2,"Diesel":3,"CNG":4,"Plug-in Hybrid":5,"LPG":6,"Hydrogen":7})
allowed_fuel_types = [1, 2, 3, 4, 5, 6, 7]
data = data[data["Fuel type"].apply(lambda x: x in allowed_fuel_types)]

# 'Hybrid' is changed to 1,
# 'Petrol' is changed to 2, 
# 'Diesel' is changed to 3, 
# 'CNG' is changed to 4, 
# 'Plug-in Hybrid' is changed to 5, 
# 'LPG' is changed to 6,
# 'Hydrogen' is changed to 7








In [3]:
# Mapping

data["Manufacturer"] = data["Manufacturer"].replace({"LEXUS":1, "CHEVROLET":2,"GREATWALL":3,"HONDA":4,"FORD":5,"HYUNDAI":6,"TOYOTA":7,
                                                     "MERCEDES-BENZ":8, "OPEL":9,"PORSCHE":10,"BMW":11,"JEEP":12,"VOLKSWAGEN":13,"AUDI":14,
                                                     "RENAULT":15, "NISSAN":16,"SUBARU":17,"DAEWOO":18,"KIA":19,"MITSUBISHI":20,"SSANGYONG":21,
                                                     "MAZDA":22, "GMC":23,"FIAT":24,"INFINITI":25,"ALFA ROMEO":26,"SUZUKI":27,"ACURA":28,
                                                     "LINCOLN":29, "VAZ":30,"GAZ":31,"CITROEN":32,"LAND ROVER":33,"MINI":34,"DODGE":35,
                                                     "CHRYSLER":36, "JAGUAR":37,"ISUZU":38,"SKODA":39,"DAIHATSU":40,"BUICK":41,"TESLA":42,
                                                     "CADILLAC":43, "PEUGEOT":44,"BENTLEY":45,"VOLVO":46,"HAVAL":47,"HUMMER":48,"SCION":49,
                                                     "UAZ":50, "MERCURY":51,"ZAZ":52,"ROVER":53,"SEAT":54,"LANCIA":55,"MOSKVICH":56,
                                                     "MASERATI":57, "FERRARI":58,"SAAB":59,"LAMBORGHINI":60,"PONTIAC":61,"SATURN":62,"ASTON MARTIN":63})


In [4]:
# Remove "KM"
data["Mileage"] = data["Mileage"].str.replace("km","")
data["Mileage"] = data["Mileage"].astype(float)

In [5]:
data["Category"] = data["Category"].replace({"Jeep":1, "Hatchback":2,"Sedan":3,"Microbus":4,"Goods wagon":5,"Universal":6,"Coupe":7,
                                             "Minivan":8, "Cabriolet":9,"Limousine":10,"Pickup":11})

In [6]:
data["Color"] = data["Color"].replace({"Silver":1, "Black":2,"White":3,"Grey":4,"Blue":5,"Green":6,"Red":7,
                                                     "Sky blue":8, "Orange":9,"Yellow":10,"Golden":11,"Beige":12,"Brown":13,"Carnelian red":14,
                                                     "Purple":15, "Pink":16})

data = pd.get_dummies(data, columns=["Color"])


In [7]:
data["Drive wheels"] = data["Drive wheels"].replace({"4x4":1, "Front":2,"Rear":3})

In [8]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
encoder = LabelEncoder()

# Fit the encoder on the "Model" feature
encoder.fit(data["Model"])

# Transform the "Model" feature to numeric
data["Model"] = encoder.transform(data["Model"])


In [9]:
Cat_types = data["Model"].unique()
p = 0
for c in Cat_types:
    p += 1  
    print(p,c)

1 1228
2 651
3 677
4 654
5 1288
6 1140
7 1315
8 431
9 1221
10 595
11 1397
12 1432
13 371
14 626
15 1223
16 569
17 794
18 838
19 1461
20 461
21 1508
22 817
23 372
24 825
25 882
26 1376
27 1148
28 258
29 657
30 477
31 1182
32 1018
33 589
34 1178
35 312
36 774
37 902
38 158
39 535
40 735
41 1428
42 206
43 1417
44 1473
45 452
46 1041
47 978
48 1546
49 521
50 531
51 1077
52 772
53 139
54 717
55 926
56 110
57 1129
58 591
59 801
60 198
61 793
62 1343
63 711
64 1132
65 89
66 669
67 247
68 609
69 928
70 918
71 481
72 175
73 320
74 1299
75 1008
76 1198
77 1094
78 1371
79 26
80 848
81 119
82 943
83 642
84 1302
85 1477
86 207
87 1047
88 1212
89 459
90 1185
91 1107
92 578
93 2
94 118
95 329
96 1475
97 85
98 226
99 1124
100 552
101 807
102 1430
103 441
104 1130
105 233
106 851
107 256
108 15
109 822
110 409
111 1102
112 1441
113 1206
114 1303
115 403
116 286
117 288
118 470
119 418
120 274
121 879
122 501
123 1248
124 993
125 151
126 790
127 1300
128 1387
129 339
130 244
131 1422
132 644
133 347
134

In [10]:
# # Assign each value of the "Model" column with the integer value of the corresponding "Manufacturer"
# data["Model_enc"] = data["Manufacturer"]


In [11]:
data.head(1100)


,ID,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Mileage,Cylinders,Gear box type,...,Color_7,Color_8,Color_9,Color_10,Color_11,Color_12,Color_13,Color_14,Color_15,Color_16
0,1,1,1228,2010,1,1,1,186005.0,6,1,...,0,0,0,0,0,0,0,0,0,0
1,2,2,651,2011,1,0,2,192000.0,6,2,...,0,0,0,0,0,0,0,0,0,0
2,3,4,677,2006,2,0,2,200000.0,4,3,...,0,0,0,0,0,0,0,0,0,0
3,4,5,654,2011,1,1,1,168966.0,4,1,...,0,0,0,0,0,0,0,0,0,0
4,5,4,677,2014,2,1,2,91901.0,4,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,1139,12,491,2009,1,1,2,163866.0,4,1,...,1,0,0,0,0,0,0,0,0,0
1139,1140,7,431,2017,3,1,2,20877.0,4,1,...,0,0,0,0,0,0,0,0,0,0
1140,1141,5,735,2013,3,1,2,137000.0,4,2,...,0,0,0,0,0,0,0,0,0,0
1142,1143,17,1542,2013,1,1,1,800.0,6,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# # Define the features and labels
# data = pd.DataFrame(data, columns=X.columns)


# X = data.drop("Price", axis=1)
# y = np.log(data["Price"])

# # fit the scaler on X
# scaler.fit(X)

# # Scale the data
# X = scaler.transform(X)

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [14]:
# NOT WORKING

#------------------------------------------------------------------------------

# Split the data into features (inputs) and target (output)
X = data.drop('Price', axis=1)
y = data['Price'].astype('float32')


# Normalize the input features
X_num = X.select_dtypes(include=['float', 'int'])
X_num = (X_num - X_num.mean()) / X_num.std()

# Shuffle the data
X_num, y = shuffle(X_num, y, random_state=42)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#Define the model
model = Sequential()

#Add layers to the model
model.add(Dense(32, input_dim=X_num.shape[1], activation='LeakyRelu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compile the model
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer=Adam(learning_rate=0.001), loss=root_mean_squared_error)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
test_loss = test_loss.ravel()[0]
print('Test loss:', test_loss)

# Save the model
model.save('used_car_price_model.h5')

# Save the training history
import pickle
with open('used_car_price_history', 'wb') as f:
    pickle.dump(history.history, f)



ValueError: Unknown activation function: LeakyRelu. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.